# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124198e+02     1.574762e+00
 * time: 0.1254270076751709
     1     1.090889e+01     9.171317e-01
 * time: 0.9730520248413086
     2    -1.189837e+01     9.784224e-01
 * time: 1.1143200397491455
     3    -3.397985e+01     7.814788e-01
 * time: 1.2465360164642334
     4    -4.743987e+01     5.291826e-01
 * time: 1.3929998874664307
     5    -5.668114e+01     2.271436e-01
 * time: 1.5340819358825684
     6    -5.942658e+01     2.252060e-01
 * time: 1.6428260803222656
     7    -6.063837e+01     9.113060e-02
 * time: 1.7450120449066162
     8    -6.123661e+01     4.940734e-02
 * time: 1.853287935256958
     9    -6.158205e+01     3.772640e-02
 * time: 1.9636669158935547
    10    -6.178257e+01     3.319147e-02
 * time: 2.0670230388641357
    11    -6.191844e+01     2.795911e-02
 * time: 2.1753599643707275
    12    -6.201881e+01     1.832868e-02
 * time: 2.283026933670044
    13    -6.209772e+01     1.356023e-02
 * time: 2.385375976562

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557653
    AtomicNonlocal      14.8522620
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666458342
